In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from datetime import datetime

In [2]:
data = pd.read_csv('movie_bd_v5.xls')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
977,tt1446714,130000000,403170142,Prometheus,Noomi Rapace|Michael Fassbender|Charlize Thero...,Ridley Scott,The Search for Our Beginning Could Lead to Our...,A team of explorers discover a clue to the ori...,124,Science Fiction|Adventure|Mystery,Twentieth Century Fox Film Corporation|Dune En...,5/30/2012,6.2,2012
1485,tt0443543,40000000,86915017,The Illusionist,Edward Norton|Jessica Biel|Paul Giamatti|Rufus...,Neil Burger,Nothing Is What It Seems.,"With his eye on a lovely aristocrat, a gifted ...",110,Fantasy|Drama|Thriller|Romance,Michael London Productions|Stillking Films|Bob...,8/18/2006,7.0,2006
1782,tt0469623,16000000,2849142,Things We Lost in the Fire,Halle Berry|Benicio del Toro|David Duchovny|Al...,Susanne Bier,Hope comes with letting go.,A recent widow invites her husband's troubled ...,113,Drama,DreamWorks SKG|Neal Street Productions|DreamWo...,9/26/2007,6.5,2007
1034,tt1430626,60000000,118338361,The Pirates! In an Adventure with Scientists!,Hugh Grant|Brendan Gleeson|Jeremy Piven|Brian ...,Peter Lord|Jeff Newitt,It's A Plunderful Life,"In The Pirates! Band of Misfits, Hugh Grant st...",88,Animation|Adventure|Family|Comedy,Columbia Pictures|Aardman Animations|Sony Pict...,3/12/2012,6.4,2012
173,tt1086772,40000000,123494610,Blended,Drew Barrymore|Adam Sandler|Wendi McLendon-Cov...,Frank Coraci,"Single Dad, No Clue. Single Mum, Flying Solo.","After a bad blind date, a man and woman find t...",117,Comedy,Happy Madison Productions|Karz Entertainment|W...,5/21/2014,6.7,2014


In [3]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [4]:
answers = {} # создадим словарь для ответов

In [5]:
# тут другие ваши предобработки колонок например:

#the time given in the dataset is in string format.
#So we need to change this in datetime format
# ...

# удалим дубликаты на всякий случай
data.drop_duplicates()

# добавляем столбец profit, вычисляется как revenue - budget
data['profit'] = data.revenue - data.budget

# превращаем столбец с жанрами в список
data.genres = data.genres.apply(lambda x: list(x.split('|')))

# режиссёров тоже
data.director = data.director.apply(lambda x: list(x.split('|')))

# ...и актёров
data.cast = data.cast.apply(lambda x: list(x.split('|')))

# и кинокомпании
data.production_companies = data.production_companies.apply(lambda x: list(x.split('|')))

# переведём дату релиза в формат datetime
data.release_date = data.release_date.apply(lambda x: datetime.strptime(x, '%m/%d/%Y'))

# и просто для удобства добавим колонку month
data['month'] = data.release_date.apply(lambda x: x.month)

# для поиска самых длинных названий вставим колонку с длиной названия
data['len_title'] = data.original_title.apply(lambda x: len(x))

# и колонку с количеством слов в описании
data['descr_words'] = data.overview.apply(lambda x: len(x.split()))

In [6]:
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,month,len_title,descr_words
625,tt0942385,92000000,188072649,Tropic Thunder,"[Ben Stiller, Robert Downey Jr., Jack Black, J...",[Ben Stiller],The movie they think they're making... isn't a...,"Vietnam veteran 'Four Leaf' Tayback's memoir, ...",107,"[Action, Comedy]","[DreamWorks SKG, Goldcrest Pictures, Red Hour ...",2008-08-09,6.4,2008,96072649,8,14,89
1556,tt0441796,9000000,27105095,Stay Alive,"[Jon Foster, Samaire Armstrong, Frankie Muniz,...",[William Brent Bell],You die in the game - you die for real.,"After the mysterious, brutal death of an old f...",100,"[Horror, Thriller]","[Spyglass Entertainment, Hollywood Pictures, E...",2006-03-24,5.3,2006,18105095,3,10,133
785,tt1655420,6400000,35057696,My Week with Marilyn,"[Michelle Williams, Eddie Redmayne, Kenneth Br...",[Simon Curtis],Sometimes You Just Need to Get Away,Sir Laurence Olivier is making a movie in Lond...,99,[Drama],"[BBC Films, The Weinstein Company, UK Film Cou...",2011-10-17,6.7,2011,28657696,10,20,96
1383,tt0342258,45000000,50871113,Unleashed,"[Jet Li, Morgan Freeman, Bob Hoskins, Vincent ...",[Louis Leterrier],Serve No Master,"Raised as a slave, Danny is used to fighting f...",103,"[Action, Crime]","[Canal Plus, Current Entertainment, Clubdeal, ...",2005-02-02,6.5,2005,5871113,2,9,77
59,tt1390411,100000000,93820758,In the Heart of the Sea,"[Chris Hemsworth, Benjamin Walker, Cillian Mur...",[Ron Howard],Based on the incredible true story that inspir...,"In the winter of 1820, the New England whaling...",122,"[Thriller, Drama, Adventure, Action, History]","[Imagine Entertainment, Spring Creek Productio...",2015-11-20,6.4,2015,-6179242,11,23,127


In [7]:
genres_count = {}
def count_titles():
    for index, row in data.iterrows():
        for genre in row.genres:
            if genre in genres_count:
                genres_count[genre] += 1
            else:
                genres_count[genre] = 1

In [8]:
count_titles()

In [9]:
unique_cast = {}
def count_actors():
    for index, row in data.iterrows():
        for actor in row.cast:
            if actor in unique_cast:
                unique_cast[actor] += 1
            else:
                unique_cast[actor] = 1

In [10]:
count_actors()

In [11]:
# украл метод вот здесь: https://towardsdatascience.com/dealing-with-list-values-in-pandas-dataframes-a177e534f173
def boolean_df(item_lists, unique_items):
# Create empty dict
    bool_dict = {}
    
    # Loop through all the tags
    for i, item in enumerate(unique_items):
        
        # Apply boolean mask
        bool_dict[item] = item_lists.apply(lambda x: item in x)
            
    # Return the results as a dataframe
    return pd.DataFrame(bool_dict)

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [14]:
# запишите свой вариант ответа
answers['1'] = '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)'
# если ответили верно, можете добавить комментарий со значком "+"

In [12]:
# можно выбрать с помощью фильтра - находим фильм, бюджет которого равен максимальному бюджету в нашем списке
print(data[data.budget == data.budget.max()].original_title)

723    Pirates of the Caribbean: On Stranger Tides
Name: original_title, dtype: object


ВАРИАНТ 2

In [13]:
# а можно отсортировать значения по столбцу "бюджет" и вывести первую строчку
print(data.sort_values(by=['budget'], ascending=False).original_title.head(1))

723    Pirates of the Caribbean: On Stranger Tides
Name: original_title, dtype: object


# 2. Какой из фильмов самый длительный (в минутах)?

In [16]:
# думаю логику работы с этим словарем вы уже поняли, 
# по этому не буду больше его дублировать
answers['2'] = '2. Gods and Generals (tt0279111)'

In [15]:
# решение идентично прошлом вопросу
print(data[data.runtime == data.runtime.max()].original_title)
# ещё интересно стало посмотреть, сколько минут он идёт)
data.iloc[[1157]].runtime

1157    Gods and Generals
Name: original_title, dtype: object


1157    214
Name: runtime, dtype: int64

# 3. Какой из фильмов самый короткий (в минутах)?





In [19]:
answers['3'] = '3. Winnie the Pooh (tt1449283)'

In [17]:
print(data[data.runtime == data.runtime.min()].original_title)
data.iloc[[768]].runtime

768    Winnie the Pooh
Name: original_title, dtype: object


768    63
Name: runtime, dtype: int64

# 4. Какова средняя длительность фильмов?


In [21]:
answers['4'] = '2. 110'

In [20]:
data.runtime.mean()

109.6585494970884

# 5. Каково медианное значение длительности фильмов? 

In [23]:
answers['5'] = '1. 107'

In [22]:
data.runtime.median()

107.0

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [25]:
answers['6'] = '5. Avatar (tt0499549)'

In [24]:
# заранее при предварительной обработке данных мы сформировали столбец "профит"
# теперь остаётся только выбрать самый прибыльный фильм так же, как мы делали раньше
print(data[data.profit == data.profit.max()].original_title)

239    Avatar
Name: original_title, dtype: object


# 7. Какой фильм самый убыточный? 

In [27]:
answers['7'] = '5. The Lone Ranger (tt1210819)'

In [26]:
print(data[data.profit == data.profit.min()].original_title)
data.iloc[[1245]]

1245    The Lone Ranger
Name: original_title, dtype: object


,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,month,len_title,descr_words
1245,tt1210819,255000000,89289910,The Lone Ranger,"[Johnny Depp, Armie Hammer, William Fichtner, ...",[Gore Verbinski],Never Take Off the Mask,The Texas Rangers chase down a gang of outlaws...,149,"[Action, Adventure, Western]","[Walt Disney Pictures, Jerry Bruckheimer Films...",2013-07-03,6.0,2013,-165710090,7,15,68


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [29]:
answers['8'] = '1. 1478'

In [28]:
# с помощью фильтра и функции count() получаем искомый ответ
data[data.revenue > data.budget].count()

imdb_id                 1478
budget                  1478
revenue                 1478
original_title          1478
cast                    1478
director                1478
tagline                 1478
overview                1478
runtime                 1478
genres                  1478
production_companies    1478
release_date            1478
vote_average            1478
release_year            1478
profit                  1478
month                   1478
len_title               1478
descr_words             1478
dtype: int64

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [31]:
answers['9'] = '4. The Dark Knight (tt0468569)'

In [30]:
# для простоты чтения кода сначала отсадим в отдельный фрейм фильмы 2008 года
# потом уже известным способом найдём самый прибыльный
data_year_2008 = data[data.release_year == 2008]
data_year_2008[data_year_2008.revenue == data_year_2008.revenue.max()].original_title

599    The Dark Knight
Name: original_title, dtype: object

In [ ]:
# (один из моих самых любимых фильмов, обожаю Нолана)

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [33]:
answers['10'] = '5. The Lone Ranger (tt1210819)'

In [32]:
data_years_2012_2014 = data[(data.release_year >= 2012) & (data.release_year <= 2014)]
print(data_years_2012_2014[data_years_2012_2014.profit == data_years_2012_2014.profit.min()].original_title)

1245    The Lone Ranger
Name: original_title, dtype: object


# 11. Какого жанра фильмов больше всего?

In [41]:
answers['11'] = '3. Drama'

In [34]:
# в предобработке написана функция, которая уже посчитала в словарь количество упоминаний разных жанров в наш фрейм
# остаётся только получить максимальное значение из словаря
max(genres_count, key=genres_count.get)

'Drama'

ВАРИАНТ 2

In [35]:
# пользуемся функцией explode - строки с несколькими значениями она превращает в несколько строк
# после этого группируем по жанрам и считаем количество значений с уникальными названиями (можно было взять другой столбец)
data_exp_genres = data.copy().explode('genres')
data_exp_genres.groupby(['genres']).original_title.count().sort_values(ascending=False)

genres
Drama              782
Comedy             683
Thriller           596
Action             582
Adventure          415
Crime              315
Romance            308
Family             260
Science Fiction    248
Fantasy            222
Horror             176
Mystery            168
Animation          139
Music               64
History             62
War                 58
Western             19
Documentary          8
Foreign              2
Name: original_title, dtype: int64

In [ ]:
ВАРИАНТ 3

In [36]:
# этот вариант честно найден на просторах гугла
# я не совсем понял эту функцию, но она позволяет применить функции pandas к списку, превратив его в series
def to_1D(series):
    return pd.Series([x for _list in series for x in _list])

In [37]:
data2 = data.copy()

In [38]:
to_1D(data2['genres']).value_counts()

Drama              782
Comedy             683
Thriller           596
Action             582
Adventure          415
Crime              315
Romance            308
Family             260
Science Fiction    248
Fantasy            222
Horror             176
Mystery            168
Animation          139
Music               64
History             62
War                 58
Western             19
Documentary          8
Foreign              2
dtype: int64

In [ ]:
ВАРИАНТ 4

In [39]:
# используем библиотеку, которую нам подсказали в шапке - Counter
# тут всё просто - создаём объект типа Counter, который считает элементы списка, который ему скормили
# счетчики можно складывать, что мы и делаеми в цикле
cnt = Counter()
for index, row in data.iterrows():
    cnt += Counter(row.genres)

In [40]:
# у счетчика есть встроенная функция вывода самых распространённых значений, что нам и нужно
cnt.most_common()

[('Drama', 782),
 ('Comedy', 683),
 ('Thriller', 596),
 ('Action', 582),
 ('Adventure', 415),
 ('Crime', 315),
 ('Romance', 308),
 ('Family', 260),
 ('Science Fiction', 248),
 ('Fantasy', 222),
 ('Horror', 176),
 ('Mystery', 168),
 ('Animation', 139),
 ('Music', 64),
 ('History', 62),
 ('War', 58),
 ('Western', 19),
 ('Documentary', 8),
 ('Foreign', 2)]

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [43]:
answers['12'] = '1. Drama'

In [42]:
# то же самое, что и в прошлом вопросе, только ещё фильтруем по условию profit > 0
data_exp_genres[data_exp_genres.profit > 0].groupby(['genres']).original_title.count().sort_values(ascending=False)

genres
Drama              560
Comedy             551
Thriller           446
Action             444
Adventure          337
Romance            242
Crime              231
Family             226
Science Fiction    195
Fantasy            188
Horror             150
Animation          120
Mystery            119
Music               47
History             46
War                 41
Western             12
Documentary          7
Name: original_title, dtype: int64

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [45]:
answers['13'] = '5. Peter Jackson'

In [44]:
# взрываем так же столбец с режиссёрами, группируем по ним же и смотрим суммы сборов
# я сознательно не выбираю первое значение из списка, потому что хочу посмотреть и другие значения
data_exp_director = data.copy().explode('director')
data_exp_director.groupby(['director']).revenue.sum().sort_values(ascending=False)

director
Peter Jackson        6490593685
Christopher Nolan    4167548502
David Yates          4154295625
Michael Bay          3886938960
J.J. Abrams          3579169916
                        ...    
David MichÃ´d           2295423
Steven Shainberg        2281089
Paul Schrader           2062066
Keanu Reeves            2054941
Simon Hunter            2033165
Name: revenue, Length: 997, dtype: int64

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [47]:
answers['14'] = '3. Robert Rodriguez'

In [46]:
# нечитабельная уже команда получается, но мы это уже делали в вопросе 12
# только в этот раз мы взорвали фрейм и по режиссёру, и по жанру
data_exp_dir_gen = data.copy().explode('genres').explode('director')
# каюсь, не разобрался как сделать explode сразу по двум колонкам
data_exp_dir_gen[data_exp_dir_gen.genres == 'Action'].groupby(['director']).original_title.count().sort_values(ascending=False)

director
Robert Rodriguez      9
Michael Bay           7
Paul W.S. Anderson    7
Ridley Scott          6
Antoine Fuqua         6
                     ..
John Whitesell        1
Brian A Miller        1
Jon Amiel             1
Jon Avnet             1
Luis Mandoki          1
Name: original_title, Length: 364, dtype: int64

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [49]:
answers['15'] = '3. Chris Hemsworth'

In [48]:
data_exp_cast = data.copy().explode('cast')
data_exp_cast[data_exp_cast.release_year == 2012].groupby(['cast']).revenue.sum().sort_values(ascending=False)

cast
Chris Hemsworth      2027450773
Denis Leary          1629460639
Anne Hathaway        1522851057
Robert Downey Jr.    1519557910
Mark Ruffalo         1519557910
                        ...    
Michael Nyqvist         3428048
Danny Huston            2106557
Josh Lucas              2106557
Sami Gayle              2106557
Nicolas Cage            2106557
Name: revenue, Length: 466, dtype: int64

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [51]:
answers['16'] = '3. Matt Damon'

In [50]:
data_high_budget = data[data.budget > data.budget.mean()]
data_exp_cast = data_high_budget.explode('cast')
data_exp_cast.groupby(['cast']).original_title.count().sort_values(ascending=False)

cast
Matt Damon                 18
Adam Sandler               17
Angelina Jolie             16
Tom Cruise                 15
Eddie Murphy               15
                           ..
Jennifer Hudson             1
Jennifer Jason Leigh        1
Jennifer Saunders           1
Jennifer Tilly              1
Ã“lafur Darri Ã“lafsson     1
Name: original_title, Length: 1505, dtype: int64

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [55]:
answers['17'] = '2. Action'

In [54]:
data_cage = data.copy().explode('genres').explode('cast')
data_cage[data_cage.cast == 'Nicolas Cage'].groupby('genres').original_title.count().sort_values(ascending=False)

genres
Action             17
Thriller           15
Drama              12
Crime              10
Fantasy             8
Adventure           7
Comedy              6
Science Fiction     4
Animation           3
Family              3
Mystery             3
History             2
Horror              1
Romance             1
War                 1
Name: original_title, dtype: int64

# 18. Самый убыточный фильм от Paramount Pictures

In [57]:
answers['18'] = '1. K-19: The Widowmaker (tt0267626)'

In [56]:
data_exp_comp = data.copy().explode('production_companies')
data_exp_comp = data_exp_comp[data_exp_comp.production_companies == 'Paramount Pictures']
data_exp_comp[data_exp_comp.profit == data_exp_comp.profit.min()].original_title

925    K-19: The Widowmaker
Name: original_title, dtype: object

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [59]:
answers['19'] = '5. 2015'

In [58]:
data.groupby(['release_year']).revenue.sum().sort_values(ascending=False)

release_year
2015    25449202382
2014    23405862953
2013    23213799791
2012    23079001687
2011    22676791872
2010    21071204961
2009    20261791024
2008    18252781990
2007    18162406801
2004    15663430720
2005    15309425558
2006    14775042320
2003    14346123312
2002    14136361487
2001    13017764865
2000    10664099805
Name: revenue, dtype: int64

# 20. Какой самый прибыльный год для студии Warner Bros?

In [61]:
answers['20'] = '1. 2014'

In [60]:
data_exp_comp = data.copy().explode('production_companies')
data_exp_comp = data_exp_comp[data_exp_comp.production_companies.str.contains('Warner Bros', na=False)]
data_exp_comp.groupby(['release_year']).profit.sum().sort_values(ascending=False)

release_year
2014    2292949646
2007    2201675217
2008    2134595031
2010    1974712985
2011    1871393682
2003    1844008221
2009    1822454136
2013    1636453400
2004    1631933725
2005    1551980298
2001    1282142579
2012    1258020056
2002    1022709901
2015     870368348
2006     625351872
2000     452631386
Name: profit, dtype: int64

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [63]:
answers['21'] = '4. Сентябрь'

In [62]:
data.groupby(['month']).original_title.count().sort_values(ascending=False)

month
9     227
12    190
10    186
8     161
3     156
4     149
6     147
11    146
7     142
5     140
2     135
1     110
Name: original_title, dtype: int64

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [66]:
answers['22'] = '2. 450'

In [64]:
data[(data.month >= 6) & (data.month <= 8)].original_title.count()

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [68]:
answers['23'] = '5. Peter Jackson'

In [67]:
# отбираем нужные месяцы, взрываем по режиссёру, группируем по режиссёру, считаем количество уникальных фильмов, сортируем по убыванию
data.copy()[(data.month == 12) | (data.month == 1) | (data.month == 2)].explode('director').groupby(['director']).original_title.count().sort_values(ascending=False)

director
Peter Jackson        7
Steven Soderbergh    6
Clint Eastwood       6
Shawn Levy           4
Adam Shankman        4
                    ..
John Luessenhop      1
John Musker          1
John Polson          1
Callan Brunker       1
Xavier Palud         1
Name: original_title, Length: 358, dtype: int64

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [71]:
answers['24'] = '5. Four By Two Productions'

In [69]:
data.copy().explode('production_companies').groupby(['production_companies']).len_title.mean().sort_values(ascending=False)

production_companies
Four By Two Productions       83.0
Jim Henson Company, The       59.0
Dos Corazones                 47.0
Polsky Films                  46.0
Museum Canada Productions     46.0
                              ... 
Everest Entertainment          3.0
Berlanti Productions           3.0
Global Entertainment Group     2.0
XM2 Productions                2.0
Ixtlan Productions             2.0
Name: len_title, Length: 1771, dtype: float64

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [73]:
answers['25'] = '3. Midnight Picture Show'

In [72]:
data.copy().explode('production_companies').groupby(['production_companies']).descr_words.mean().sort_values(ascending=False)

production_companies
Midnight Picture Show                    175.0
Room 9 Entertainment                     161.0
Heineken Branded Entertainment           159.0
98 MPH Productions                       159.0
Brookwell-McNamara Entertainment         156.0
                                         ...  
Henceforth                                13.0
Our Stories Films                         13.0
London Boulevard                          13.0
Motion Picture Corporation of America     11.0
Empire Pictures                           11.0
Name: descr_words, Length: 1771, dtype: float64

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [75]:
answers['26'] = '1. Inside Out, The Dark Knight, 12 Years a Slave'

In [74]:
# тут просто посмотрим 18 лучших фильмов и сделаем выводы
data.sort_values(by='vote_average', ascending=False).head(19)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,month,len_title,descr_words
599,tt0468569,185000000,1001921825,The Dark Knight,"[Christian Bale, Michael Caine, Heath Ledger, ...",[Christopher Nolan],Why So Serious?,Batman raises the stakes in his war on crime. ...,152,"[Drama, Action, Crime, Thriller]","[DC Comics, Legendary Pictures, Warner Bros., ...",2008-07-16,8.1,2008,816921825,7,15,67
118,tt0816692,165000000,621752480,Interstellar,"[Matthew McConaughey, Jessica Chastain, Anne H...",[Christopher Nolan],Mankind was born on Earth. It was never meant ...,Interstellar chronicles the adventures of a gr...,169,"[Adventure, Drama, Science Fiction]","[Paramount Pictures, Legendary Pictures, Warne...",2014-11-05,8.0,2014,456752480,11,12,35
125,tt2084970,14000000,233555708,The Imitation Game,"[Benedict Cumberbatch, Keira Knightley, Matthe...",[Morten Tyldum],The true enigma was the man who cracked the code.,Based on the real life story of legendary cryp...,113,"[History, Drama, Thriller, War]","[Black Bear Pictures, Bristol Automotive]",2014-11-14,8.0,2014,219555708,11,18,46
9,tt2096673,175000000,853708609,Inside Out,"[Amy Poehler, Phyllis Smith, Richard Kind, Bil...",[Pete Docter],Meet the little voices inside your head.,"Growing up can be a bumpy road, and it's no ex...",94,"[Comedy, Animation, Family]","[Walt Disney Pictures, Pixar Animation Studios...",2015-06-09,8.0,2015,678708609,6,10,112
34,tt3170832,6000000,35401758,Room,"[Brie Larson, Jacob Tremblay, Joan Allen, Sean...",[Lenny Abrahamson],Love knows no boundaries,Jack is a young boy of 5 years old who has liv...,117,"[Drama, Thriller]","[Element Pictures, No Trace Camping, A24, Dupe...",2015-10-16,8.0,2015,29401758,10,4,49
1183,tt0993846,100000000,392000694,The Wolf of Wall Street,"[Leonardo DiCaprio, Jonah Hill, Margot Robbie,...",[Martin Scorsese],EARN. SPEND. PARTY.,A New York stockbroker refuses to cooperate in...,180,"[Crime, Drama, Comedy]","[Paramount Pictures, Appian Way, EMJAG Product...",2013-12-25,7.9,2013,292000694,12,23,29
128,tt2267998,61000000,369330363,Gone Girl,"[Ben Affleck, Rosamund Pike, Carrie Coon, Neil...",[David Fincher],You don't know what you've got 'til it's...,With his wife's disappearance having become th...,145,"[Mystery, Thriller, Drama]","[Twentieth Century Fox Film Corporation, Regen...",2014-10-01,7.9,2014,308330363,10,9,30
1191,tt2024544,20000000,187000000,12 Years a Slave,"[Chiwetel Ejiofor, Michael Fassbender, Lupita ...",[Steve McQueen],The extraordinary true story of Solomon Northup,"In the pre-Civil War United States, Solomon No...",134,"[Drama, History]","[Plan B Entertainment, Regency Enterprises, Ri...",2013-10-18,7.9,2013,167000000,10,16,61
119,tt2015381,170000000,773312399,Guardians of the Galaxy,"[Chris Pratt, Zoe Saldana, Dave Bautista, Vin ...",[James Gunn],All heroes start somewhere.,"Light years from Earth, 26 years after being a...",121,"[Action, Science Fiction, Adventure]","[Marvel Studios, Moving Picture Company (MPC),...",2014-07-30,7.9,2014,603312399,7,23,28
1081,tt0167260,94000000,1118888979,The Lord of the Rings: The Return of the King,"[Elijah Wood, Ian McKellen, Viggo Mortensen, L...",[Peter Jackson],The eye of the enemy is moving.,Aragorn is revealed as the heir to the ancient...,201,"[Adventure, Fantasy, Action]","[WingNut Films, New Line Cinema]",2003-12-01,7.9,2003,1024888979,12,45,45


# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [83]:
answers['27'] = '5. Daniel Radcliffe & Rupert Grint'

In [76]:
# здесь было сложно, буду описывать по шагам
# решение опять же частично взято отсюда: https://towardsdatascience.com/dealing-with-list-values-in-pandas-dataframes-a177e534f173

# делаем копию данных и взрываем по актерам
data_exp_cast = data.copy().explode('cast')

In [77]:
# пользуемся функцией описанной в начале блокнота
cast_bool = boolean_df(data['cast'], unique_cast.keys())

In [78]:
# переводим булевы значения в целые
cast_int = cast_bool.astype(int)

In [79]:
# получаем матрицу пересечений актёров
cast_freq_mat = np.dot(cast_int.T, cast_int)

In [80]:
# забиваем главную диагональ нулями, чтобы не учитывать, сколько раз актёр снимался сам с собой
np.fill_diagonal(cast_freq_mat, 0)

In [81]:
# превращаем эту матрицу обратно в датафрейм
cast_freq = pd.DataFrame(cast_freq_mat, columns = unique_cast.keys(), index = unique_cast.keys())

In [82]:
# посмотрим, у кого из актёров есть самые большие количества пересечений с другими (пока нам неизвестными) актёрами...
cast_freq.max().sort_values(ascending=False)

Emma Watson         8
Rupert Grint        8
Daniel Radcliffe    8
Ben Stiller         6
Johnny Depp         6
                   ..
Thomas Dekker       1
Mickey O'Keefe      1
Sean Combs          1
Chris Riggi         1
Tess Harper         1
Length: 3406, dtype: int32

In [ ]:
# и теперь ответ понятен

# Submission

In [84]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': '2. Gods and Generals (tt0279111)',
 '3': '3. Winnie the Pooh (tt1449283)',
 '4': '2. 110',
 '5': '1. 107',
 '6': '5. Avatar (tt0499549)',
 '7': '5. The Lone Ranger (tt1210819)',
 '8': '1. 1478',
 '9': '4. The Dark Knight (tt0468569)',
 '10': '5. The Lone Ranger (tt1210819)',
 '11': '3. Drama',
 '12': '1. Drama',
 '13': '5. Peter Jackson',
 '14': '3. Robert Rodriguez',
 '15': '3. Chris Hemsworth',
 '16': '3. Matt Damon',
 '17': '2. Action',
 '18': '1. K-19: The Widowmaker (tt0267626)',
 '19': '5. 2015',
 '20': '1. 2014',
 '21': '4. Сентябрь',
 '22': '2. 450',
 '23': '5. Peter Jackson',
 '24': '5. Four By Two Productions',
 '25': '3. Midnight Picture Show',
 '26': '1. Inside Out, The Dark Knight, 12 Years a Slave',
 '27': '5. Daniel Radcliffe & Rupert Grint'}

In [85]:
# и убедиться что ни чего не пропустил)
len(answers)

27